In [1]:
import cProfile
import lysis
import math
import numpy as np

In [2]:
e = lysis.util.Experiment(r'..\..\data', experiment_code='2022-12-27-1100')
e.read_file()

In [3]:
macro = lysis.MacroscaleRun(e)

In [ ]:
cProfile.run(macro.run())

In [ ]:
print(e)

In [4]:
rng = np.random.default_rng(seed=24375893)

In [7]:
math.log(rng.random())

-0.42198885699956534

In [ ]:
fg = lysis.EdgeGrid(4, 5, 0, (lysis.util.BoundaryCondition.CONTINUING, lysis.util.BoundaryCondition.REFLECTING))

In [ ]:
[fg.neighbor(1, 6, k) for k in range(8)]

In [ ]:
lysis.to_fortran_edge_index(0, 3, 4, 5)

In [ ]:
lysis.from_fortran_edge_index(51, 4, 5)

In [ ]:
import numpy as np

In [ ]:
a = 3.5*np.ones((3, 4), dtype=np.double)
a[1][2]

In [ ]:
b = a[:2,:2]
b

In [ ]:
b[0,0] = 0
b

In [ ]:
a

In [ ]:
from enum import Enum, IntEnum, unique

In [ ]:
@unique
class Color(IntEnum):
    BLUE = 0
    RED = 1
    GREEN = 2
    
@unique
class Direction(Enum):
    UP = 1
    DOWN = -1

In [ ]:
Color.RED == Color.GREEN

In [ ]:
Color.RED == Direction.UP

In [ ]:
c = Color

In [ ]:
c.RED

In [ ]:
t = (Direction.UP, Direction.UP, Direction.DOWN)

In [ ]:
t[Color.BLUE]

In [ ]:
type(Direction.UP)

In [ ]:
isinstance(Direction.UP, Direction)

In [ ]:
import sys

In [ ]:
import lysis

In [10]:
a = np.ones((5, 10))
a

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [11]:
a[:3] = 3

In [12]:
a

array([[3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
       [3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
       [3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [15]:
A = np.empty((5, 10), dtype=object)
for i, j in np.ndindex(5, 10):
    A[i, j] = []

In [16]:
A[4, 3].append(4)

In [17]:
A

array([[list([]), list([]), list([]), list([]), list([]), list([]),
        list([]), list([]), list([]), list([])],
       [list([]), list([]), list([]), list([]), list([]), list([]),
        list([]), list([]), list([]), list([])],
       [list([]), list([]), list([]), list([]), list([]), list([]),
        list([]), list([]), list([]), list([])],
       [list([]), list([]), list([]), list([]), list([]), list([]),
        list([]), list([]), list([]), list([])],
       [list([]), list([]), list([]), list([4]), list([]), list([]),
        list([]), list([]), list([]), list([])]], dtype=object)

In [5]:
import numpy as 
rng = np.random.default_rng(seed=5748394)

In [4]:
a = cp.empty(10000)
%timeit rng.random(out=a)

55.2 µs ± 23.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
a = cp.empty(10000)
%timeit for i in cp.arange(a.size): a[i] = rng.random()

656 ms ± 42.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
float(rng.random())

0.8534137010574341

In [2]:
import lysis
import math
import numpy as np
import cupy as cp

In [2]:
e = lysis.util.Experiment(r'../../data', experiment_code='2022-12-27-1100')
e.initialize_macro_param()
macro = lysis.MacroscaleRun(e)

In [3]:
def find_binding_time(macro, current_time: float) -> float:
    binding_time_interval = -math.log(macro.rng.random()) / (macro.exp.macro_params.binding_rate
                                                            * macro.exp.macro_params.binding_sites)
    return current_time + binding_time_interval - macro.exp.macro_params.time_step / 2

In [6]:
%%timeit
for i in np.arange(1000000):
    find_binding_time(macro, 10.0*i)

3.19 s ± 17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%%timeit
for i in np.arange(1000000):
    macro.find_binding_time(10.0*i)

2.16 s ± 14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
rng = cp.random.default_rng(seed=54534534)